In [ ]:
# Conversation buffer memory: keeps every token in the past
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    # return_messages=True
)

memory.save_context({"input": "Hi"}, {"output": "How are you?"})

memory.load_memory_variables({})

In [ ]:
# Conversation buffer window memory: keeps past conversation window of length k
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
  return_messages=True,
  k=4,
)

def add_messages(input, output):
  memory.save_context({"input": input}, {"output": output})

add_messages(1, 1)
add_messages(2, 2)
add_messages(3, 3)
add_messages(4, 4)

memory.load_memory_variables({})

In [ ]:
add_messages(5, 5)

memory.load_memory_variables({})

In [ ]:
# Conversation summary memory
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOllama


chat = ChatOllama(model="mistral:latest")

memory = ConversationSummaryMemory(llm=chat)

def get_history():
  return memory.load_memory_variables({})

add_messages("Hi I'm Son, I live in South Korea", "Whoa, that is so cool!")


In [ ]:
add_messages("South Korea is so beautiful.", "I wish I could go!")

In [ ]:
get_history()

In [ ]:
# Conversation summary buffer memory: mix of summary for old conversations and buffer memory of recent ones
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOllama


chat = ChatOllama(model="mistral:latest")

memory = ConversationSummaryBufferMemory(
  return_messages=True,
  llm=chat,
  max_token_limit=150, # max number of tokens before the buffer memory gets summarized
)

add_messages("Hi I'm Son, I live in South Korea", "Whoa, that is so cool!")

get_history()

In [ ]:
add_messages("South Korea is so beautiful.", "I wish I could go!")

get_history()

In [ ]:
add_messages("How far is South Korea from Argentine?", "I don't know! Maybe super far!!")

get_history()

In [ ]:
add_messages("How far is Brazil from Argentine?", "I don't know! Maybe super near!!")

get_history()

In [ ]:
# Conversation Knowledge Graph memory: it builds a knowledge graph of entities from the conversation
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOllama


chat = ChatOllama(model="mistral:latest")

memory = ConversationKGMemory(
  llm=chat,
  return_messages=True,
)

add_messages("Hi I'm Son, I live in South Korea", "Who is Son?")
add_messages("Son is your friend.", "Okay")

In [ ]:
memory.load_memory_variables({"input": "Who is Son?"})

In [ ]:
add_messages("Son likes Kimchi.", "What does Son like?")
add_messages("Son likes Kimchi.", "Okay")

In [ ]:
memory.load_memory_variables({"input": "What does Son like?"})

In [ ]:
# Memory on LLMChain
from langchain.chat_models import ChatOllama
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


chat = ChatOllama(model="mistral:latest")

memory = ConversationSummaryBufferMemory(
  llm=chat,
  max_token_limit=120,
  memory_key="chat_history"
)

template = """
  You are a helpful AI talking to a human.

  {chat_history}
  Human: {question}
  You:
"""

chain = LLMChain(
  llm=chat,
  memory=memory,
  prompt=PromptTemplate.from_template(template),
  verbose=True
)

chain.predict(question="My name is Son.")

In [ ]:
chain.predict(question="I live in Seoul.")

In [ ]:
chain.predict(question="What is my name?")

In [ ]:
memory.load_memory_variables({})

In [ ]:
# Chat based memory
from langchain.chat_models import ChatOllama
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


chat = ChatOllama(model="mistral:latest")

memory = ConversationSummaryBufferMemory(
  llm=chat,
  max_token_limit=120,
  memory_key="chat_history",
  return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful AI talking to a human."),
  MessagesPlaceholder(variable_name="chat_history"),
  ("human", "{question}"),
])

chain = LLMChain(
  llm=chat,
  memory=memory,
  prompt=prompt,
  verbose=True
)

chain.predict(question="My name is Son.")

In [1]:
# LCEL based memory
from langchain.chat_models import ChatOllama
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


chat = ChatOllama(model="mistral:latest")

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=120,
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

def load_memory(_):
  return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | chat

def invoke_chain(question):
  result = chain.invoke({"question": question})
  memory.save_context({"input": question}, {"output": result.content})
  print(result)

In [2]:
invoke_chain("My name is Son.")

/opt/homebrew/anaconda3/envs/fs-gpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


content=" Hello Son, it's nice to meet you. How can I assist you today? If you have any questions or need help with something specific, feel free to ask."


In [3]:
invoke_chain("What is my name?")

content=" Your name is Son, as you mentioned earlier in our conversation. Is there anything particular you'd like to know or discuss related to that name or any other topic? I'm here to help answer any questions you might have."
